In [1]:
!pip3 install mlforecast
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 844.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 13.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0


In [4]:
# Load data
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

path1 = 'Sign1_full_fitted.csv'
data1 = pd.read_csv(path1, usecols=[0,1], header=0)
print(data1)


                        ts    y1
0      2016-11-09 13:03:00   0.0
1      2016-11-09 13:04:00   0.0
2      2016-11-09 13:05:00   0.0
3      2016-11-09 13:06:00   0.0
4      2016-11-09 13:07:00   0.0
...                    ...   ...
12892  2016-11-18 11:55:00  82.0
12893  2016-11-18 11:56:00  79.0
12894  2016-11-18 11:57:00  84.0
12895  2016-11-18 11:58:00  88.0
12896  2016-11-18 11:59:00  87.0

[12897 rows x 2 columns]


In [5]:
path2 = 'Sign12_full_fitted.csv'
data2 = pd.read_csv(path2, usecols=[0,1], header=0)
print(data2)

                        ts    y12
0      2016-11-09 13:03:00    0.0
1      2016-11-09 13:04:00    0.0
2      2016-11-09 13:05:00    0.0
3      2016-11-09 13:06:00    0.0
4      2016-11-09 13:07:00    0.0
...                    ...    ...
12892  2016-11-18 11:55:00  340.0
12893  2016-11-18 11:56:00  340.0
12894  2016-11-18 11:57:00  336.0
12895  2016-11-18 11:58:00  334.0
12896  2016-11-18 11:59:00  335.0

[12897 rows x 2 columns]


In [6]:
path4 = 'Sign14_full_fitted.csv'
data4 = pd.read_csv(path4, usecols=[0,1], header=0)
print(data4)

                        ts    y14
0      2016-11-09 13:03:00   20.0
1      2016-11-09 13:04:00   20.0
2      2016-11-09 13:05:00   20.0
3      2016-11-09 13:06:00   16.0
4      2016-11-09 13:07:00   16.0
...                    ...    ...
12892  2016-11-18 11:55:00  333.0
12893  2016-11-18 11:56:00  334.0
12894  2016-11-18 11:57:00  335.0
12895  2016-11-18 11:58:00  336.0
12896  2016-11-18 11:59:00  335.0

[12897 rows x 2 columns]


In [13]:
train = data1.loc[:,:]
valid = data2.loc[:,:]

In [14]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

models = [RandomForestRegressor(random_state=0, n_estimators=100),
          ExtraTreesRegressor(random_state=0, n_estimators=100)]

In [15]:
from numba import njit

@njit
def diff(x, lag):
    x2 = np.full_like(x, np.nan)
    for i in range(lag, len(x)):
        x2[i] = x[i] - x[i-lag]
    return x2

In [18]:
from window_ops.rolling import rolling_mean
from mlforecast import MLForecast

model = MLForecast(models=models,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 3), (rolling_mean, 7), (rolling_mean, 28), (diff, 1), (diff, 7)],
                   },
                   date_features=['dayofweek'],
                   num_threads=6)